In [18]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import optuna

# Настройки отображения
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:.2f}'.format

# Инициализация путей
root = Path(os.getcwd()).parent.parent
data_in = root / 'data' / 'eda_data'
data_out = root / 'data' / 'model_data'

print("Пути инициализированы:")
print(f"EDA data: {data_in}")
print(f"Model data: {data_out}")

Пути инициализированы:
EDA data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/eda_data
Model data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/model_data


In [20]:
# Загрузка данных
print("\nЗагружаем предобработанные данные...")
data = pd.read_pickle(data_in / 'eda_data.pkl')
print("\nДанные успешно загружены!")


Загружаем предобработанные данные...

Данные успешно загружены!


In [21]:
 def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    
    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())
    
    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")
        
    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")
    
    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def analyze_column(df: pd.DataFrame, col: str, max_display: int = 50) -> None:
    """Полный анализ колонки с выводом всех уникальных значений"""
    print(f"\n{'-'*60}")
    print(f"Полный анализ колонки: {col}")
    
    # Проверка существования колонки
    if col not in df.columns:
        print(f"Колонка {col} не найдена!")
        return
    
    # Пропуски
    na_count = df[col].isna().sum()
    print(f"Пропуски: {na_count} ({na_count/len(df)*100:.1f}%)")

    # Количество уникальных значений
    unique_count = df[col].nunique(dropna=False)
    print(f"Уникальных значений: {unique_count}")
    
    # Вывод всех значений для категориальных данных
    if unique_count <= max_display:
        print("\nВсе значения:")
        print(df[col].unique())
    else:
        print(f"\nСлишком много значений (> {max_display}). Примеры:")
        print(df[col].dropna().sample(10).unique())
        
    # Частотный анализ для числовых колонок
    if pd.api.types.is_numeric_dtype(df[col]):
        print("\nОписательная статистика:")
        print(df[col].describe())
    else:
        print("\nТоп-10 значений:")
        print(df[col].value_counts(dropna=False).head(10))

In [22]:
generate_summary(data, "Данные")
for col in data.columns:
    analyze_column(data, col, max_display=100)


================================================== ДАННЫЕ ==================================================
Общее количество записей: 13,332,940
Количество признаков: 30

Типы данных:
         count
int32       14
object      13
float32      3

Пропущенных значений нет

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_height,device_screen_width,device_browser,geo_country,geo_city,hit_number,is_target,aspect_ratio,n_hits,n_target_hits,n_unique_pages,visit_day,visit_month,visit_year,hit_sec,session_total_sec
0,9055434745589932991.163775...,2108382700.1637753791,14,2,0,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,other_russia_city,3,0,0.50,4,1,1,24,11,2021,3.66,42.93
1,9055434745589932991.163775...,2108382700.1637753791,14,2,0,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,other_russia_city,4,1,0.50,4,1,1,24,11,2021,46.59,42.93
2,905544597018549464.1636867...,210838531.1636867288,8,6,1,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,854,385,Samsung Internet,russia,Moscow,3,0,0.45,3,0,1,14,11,2021,0.92,0.00



------------------------------------------------------------
Полный анализ колонки: session_id
Пропуски: 0 (0.0%)
Уникальных значений: 1582604

Слишком много значений (> 100). Примеры:
['2055298537470059424.1632134048.1632134048'
 '3039519408930726922.1631101961.1631101961'
 '1773969319066290256.1633896529.1633896529'
 '8999281441221843248.1640700209.1640700209'
 '2358498511039615552.1640878656.1640878656'
 '6302432847118332700.1631449886.1631449886'
 '5388377877123593969.1636202009.1636202009'
 '6021740572356741881.1625393913.1625393913'
 '7267888147478318529.1640172690.1640172690'
 '4019599055753681105.1639028943.1639028943']

Топ-10 значений:
session_id
5692861315757623740.1632356796.1632356796    120
1544572560279928739.1632436140.1632436140    120
477529334778357839.1632299092.1632299092     116
6766315690481471549.1632423998.1632423998    116
6556325878862444653.1632368752.1632368752    116
4781656099508856169.1632511340.1632511340    116
6344040849605405862.1632382118.163238211

In [28]:
df = data.copy()

cat_features = [
    'visit_hour', 'visit_weekday', 'is_weekend', 'visit_number',
    'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword',
    'device_category', 'device_os', 'device_brand', 'device_browser',
    'geo_country', 'geo_city'
]

# Отдельный таргет
X = df.drop(['session_id', 'client_id', 'is_target'], axis=1)
y = df['is_target'].astype(int)

# Проверяем, что они есть в X
cat_features = [c for c in cat_features if c in X.columns]

# Разбиение на train/val
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Создаём Pool-ы для CatBoost
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool   = Pool(data=X_val,   label=y_val,   cat_features=cat_features)

# Optuna-целевая функция
def objective(trial):
    params = {
        'iterations':            trial.suggest_int('iterations', 200, 800),
        'learning_rate':         trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
        'depth':                 trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg':           trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10),
        'border_count':          trial.suggest_int('border_count', 32, 128),
        # настройки для Mac M1 (CPU-потоки)
        'task_type':             'CPU',
        'thread_count':          4,
        'random_seed':           42,
        'eval_metric':           'AUC',
        'verbose':               False,
        'early_stopping_rounds': 50,
    }
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)
    pred = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, pred)

# Запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, timeout=1800)  # 30 испытаний или 30 мин

print(">>> Best AUC:", study.best_value)
print(">>> Best params:", study.best_params)

# Финальное обучение на всей выборке
best_params = study.best_params
best_params.update({
    'task_type':             'CPU',
    'thread_count':          4,
    'random_seed':           42,
    'eval_metric':           'AUC',
    'verbose':               100,
})
final_model = CatBoostClassifier(**best_params)
full_pool   = Pool(data=X, label=y, cat_features=cat_features)
final_model.fit(full_pool)

# Сохраняем модель
os.makedirs(data_out, exist_ok=True)
model_path = data_out / 'catboost_optuna_m1.cbm'
final_model.save_model(str(model_path))

print(f"Модель сохранена в {model_path}")

[I 2025-05-06 20:21:38,495] A new study created in memory with name: no-name-1da026e9-d8cf-4d02-8c3a-7ff76c7cd6d9
/var/folders/hn/tkf0_38d2d1ccs6db5_mfvmw0000gn/T/ipykernel_1509/3673298917.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':         trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
/var/folders/hn/tkf0_38d2d1ccs6db5_mfvmw0000gn/T/ipykernel_1509/3673298917.py:32: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg':           trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10),
[I 2025-05-06 20:47:13,179] Trial 0 finished with value: 0.8701588599195377 and parameters: {'iterations': 697, 'learning_rate': 0.001994017704749788, 'd

>>> Best AUC: 0.8951835898088527
>>> Best params: {'iterations': 523, 'learning_rate': 0.010456644170233342, 'depth': 8, 'l2_leaf_reg': 0.0870186749348508, 'border_count': 125}
0:	total: 19s	remaining: 2h 45m 7s
100:	total: 36m 50s	remaining: 2h 33m 55s
200:	total: 1h 10m 24s	remaining: 1h 52m 47s
300:	total: 1h 42m 49s	remaining: 1h 15m 49s
400:	total: 2h 14m 43s	remaining: 40m 59s
500:	total: 2h 48m 11s	remaining: 7m 23s
522:	total: 2h 56m 2s	remaining: 0us
Модель сохранена в /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/model_data/catboost_optuna_m1.cbm


In [30]:
from sklearn.metrics import roc_auc_score
# Оценка на валидации
val_pred = final_model.predict_proba(X_val)[:, 1]
print("ROC-AUC на валидации:", roc_auc_score(y_val, val_pred))


ROC-AUC на валидации: 0.8951076056582663
